# 🎯 Attention-LSTM Training Notebook

This notebook loads historical price data from all asset classes, prepares time-series sequences, trains an Attention-based LSTM model, and saves it as `trained-attention-lstm.pth`.


## [Block 1] Imports

In [1]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

## [Block 2] Attention-Based LSTM Model

In [2]:
class AttentionLSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, output_dim=1):
        super(AttentionLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.attn = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def attention(self, lstm_output):
        weights = torch.softmax(self.attn(lstm_output).squeeze(-1), dim=1)
        context = torch.bmm(weights.unsqueeze(1), lstm_output).squeeze(1)
        return context

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        context = self.attention(lstm_out)
        return self.fc(context)

## [Block 3] Load Historical Data from All Asset Classes

In [3]:
def load_all_time_series(data_root='data/historical-data', seq_len=60):
    X, y = [], []
    for root, _, files in os.walk(data_root):
        for file in files:
            if file.endswith('.csv'):
                df = pd.read_csv(os.path.join(root, file))
                if 'Close' not in df.columns or len(df) < seq_len + 1:
                    continue
                df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
                df.dropna(inplace=True)
                series = df['log_return'].values[-(seq_len + 1):]
                scaled = MinMaxScaler().fit_transform(series.reshape(-1, 1)).flatten()
                X.append(scaled[:-1])
                y.append(scaled[-1])
    X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
    y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)
    return X_tensor, y_tensor

## [Block 4] Train the Model

In [4]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Accuracy metric: % of predictions within ±10% of actual value
def percentage_accuracy(y_true, y_pred, tolerance=0.10):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    within_tolerance = np.abs(y_true - y_pred) <= (tolerance * np.abs(y_true))
    return np.mean(within_tolerance) * 100

# Load and process time series from all assets
def load_all_time_series(data_root, seq_len=60):
    X, y = [], []
    for root, _, files in os.walk(data_root):
        for file in files:
            if file.endswith('.csv'):
                df = pd.read_csv(os.path.join(root, file))

                if 'Close' not in df.columns or len(df) < seq_len + 2:
                    continue

                # Ensure 'Close' is numeric
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

                # 🔁 Calculate log returns safely
                df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
                df['log_return'] = df['log_return'].replace([np.inf, -np.inf], np.nan)
                df.dropna(subset=['log_return'], inplace=True)

                series = df['log_return'].values[-(seq_len + 1):]
                if len(series) < seq_len + 1:
                    continue

                scaled = MinMaxScaler().fit_transform(series.reshape(-1, 1)).flatten()
                X.append(scaled[:-1])
                y.append(scaled[-1])
    
    # Use np.array() to avoid torch tensor warnings
    X_tensor = torch.tensor(np.array(X), dtype=torch.float32).unsqueeze(-1)
    y_tensor = torch.tensor(np.array(y), dtype=torch.float32).unsqueeze(-1)
    return X_tensor, y_tensor

X, y = load_all_time_series(data_root='./data/historical-data')
print("Training samples:", X.shape[0])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)

# ⚙️ Model, loss, optimizer
model = AttentionLSTM()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

best_val_accuracy = 0

# Training loop
for epoch in range(100):
    model.train()
    total_loss = 0
    all_train_preds = []
    all_train_targets = []

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_X)
        loss = loss_fn(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        all_train_preds.extend(output.detach().cpu().numpy())
        all_train_targets.extend(batch_y.detach().cpu().numpy())

    # Validation
    model.eval()
    val_preds, val_targets = [], []
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            output = model(batch_X)
            val_preds.extend(output.cpu().numpy())
            val_targets.extend(batch_y.cpu().numpy())

    train_acc = percentage_accuracy(all_train_targets, all_train_preds)
    val_acc = percentage_accuracy(val_targets, val_preds)

    if epoch % 10 == 0:
        print(f"Epoch {epoch:02d} - Loss: {total_loss:.6f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc

# Final result
print(f"\n-> Final Validation Accuracy: {best_val_accuracy:.2f}%")


/opt/miniconda3/envs/finsight/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/miniconda3/envs/finsight/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/miniconda3/envs/finsight/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/miniconda3/envs/finsight/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/miniconda3/envs/finsight/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/miniconda3/envs/finsight/lib/python

Training samples: 544
Epoch 00 - Loss: 3.751016 | Train Acc: 0.00% | Val Acc: 0.00%
Epoch 10 - Loss: 0.513614 | Train Acc: 22.53% | Val Acc: 27.52%
Epoch 20 - Loss: 0.486272 | Train Acc: 24.60% | Val Acc: 27.52%
Epoch 30 - Loss: 0.453530 | Train Acc: 29.43% | Val Acc: 33.94%
Epoch 40 - Loss: 0.428097 | Train Acc: 34.71% | Val Acc: 33.03%
Epoch 50 - Loss: 0.432132 | Train Acc: 35.86% | Val Acc: 38.53%
Epoch 60 - Loss: 0.412836 | Train Acc: 34.71% | Val Acc: 27.52%
Epoch 70 - Loss: 0.430031 | Train Acc: 32.64% | Val Acc: 33.03%
Epoch 80 - Loss: 0.426731 | Train Acc: 35.17% | Val Acc: 44.04%
Epoch 90 - Loss: 0.406952 | Train Acc: 37.47% | Val Acc: 38.53%

-> Final Validation Accuracy: 46.79%


## [Block 5] Save the Trained Model

In [5]:
torch.save(model.state_dict(), 'trained-attention-lstm.pth')
print(" Model saved as trained-attention-lstm.pth")

 Model saved as trained-attention-lstm.pth
